In [137]:
import matplotlib.pyplot as plt
import numpy as np
import time
from cvxpy import *
from scipy.interpolate import interp1d

NUM_CARS = 4
CONTROL_HORIZON = 24*6
SIMULATION_TIME = 24*6


def get_parameters():
    alpha = 20
    beta = 0.1
    gamma = 1
    u_max = 0.1
    u_tot_max = 0.03
    return alpha, beta, gamma, u_max, u_tot_max

def get_data():
    #prices = np.array([1, 1, 1, 1, 8, 9, 8, 8, 6, 5, 4, 4, 5, 5, 6, 6, 7, 10, 12, 7, 5, 3, 2, 1])
    prices_day = np.array([1, 1, 1, 1, 8, 9, 8, 8, 6, 5, 4, 4, 5, 5, 6, 6, 7, 10, 12, 7, 5, 3, 2, 1])
    repeat=SIMULATION_TIME/len(prices_day)
    prices=[]
    for i in range(repeat):
        prices = np.concatenate((prices, prices_day))
    prices = np.concatenate((prices, [0]))

    
    #prices = np.concatenate((prices, [0]))
    cars_presence = np.ones((NUM_CARS, SIMULATION_TIME))
    for  j in range(SIMULATION_TIME-50):
        cars_presence[1, j] = 0

    return prices, cars_presence
get_data()[0].shape
#print get_data()[0].shape
#print get_data()[1].shape


(145L,)

In [146]:
def get_model_matrix():
    gamma = get_parameters()[2]
    cars_presence = get_data()[1]

    A = np.zeros((NUM_CARS, NUM_CARS, SIMULATION_TIME))
    B = np.zeros((NUM_CARS, NUM_CARS, SIMULATION_TIME))

    for i in range(SIMULATION_TIME):
        A[:, :, i] = np.eye(NUM_CARS)
        B[:, :, i] = np.eye(NUM_CARS) * gamma

    for i in range(NUM_CARS):
        A[i, i, :] = np.multiply(A[i, i, :], cars_presence[i, :])
        B[i, i, :] = np.multiply(B[i, i, :], cars_presence[i, :])

    return A, B

def get_cost_matrix():
    prices, cars_presence = get_data()

    C=np.zeros((NUM_CARS, SIMULATION_TIME))

    for i in range(NUM_CARS):
        C[i,:]= prices[:-1].reshape((1,len(prices)-1))
    return C


In [159]:
def mpc_control(x0):
    x = cvxpy.Variable(NUM_CARS, CONTROL_HORIZON+1)
    u = cvxpy.Variable(NUM_CARS, CONTROL_HORIZON)

    prices, cars_presence = get_data()

    C = get_cost_matrix()

    alpha, beta, gamma, u_max, u_tot_max = get_parameters()
    print (cars_presence).shape
    cost = 0
    constraints = []
    
    for i in range(NUM_CARS):
        cost+=x[i,:]*prices.reshape((CONTROL_HORIZON+1,1))
        for t in range(CONTROL_HORIZON-1):
            constraints += [x[i, t+1] ==  x[i, t] +   u[i, t]*cars_presence[i,t] ]
            constraints += [u[i, t] >= 0, u[i, t] <= np.ones((NUM_CARS)) - x[i, t]]
            constraints += [u[i, t] <= u_max]
            constraints += [np.sum(u[i, t]) <= u_tot_max]
    constraints += [x[:, 0] == x0]

    prob = cvxpy.Problem(cvxpy.Minimize(cost), constraints)

    start = time.time()
    #prob.solve(verbose=False)
    prob.solve(solver=SCS)
    elapsed_time = time.time() - start
    print('calculation time: {0} [sec]'.format(elapsed_time))

    if prob.status == cvxpy.OPTIMAL:
        return x.value


if __name__ == '__main__':
    x = mpc_control(np.array([0.5, 0.2, 0, 0.1]))
    print 'x is ', x
    prices, cars_presence  = get_data()
    plt.figure(figsize=(10, 7))
    for i in range(NUM_CARS):
        plt.plot(range(CONTROL_HORIZON+1), x[i, :].T, label='soc of car '+str(i))
    plt.plot(range(CONTROL_HORIZON), prices[:CONTROL_HORIZON]/max(prices), label='prices')
    plt.legend()
    plt.grid()
    plt.show()


(4L, 144L)
WARN: A->p (column pointers) not strictly increasing, column 576 empty
WARN: A->p (column pointers) not strictly increasing, column 577 empty
WARN: A->p (column pointers) not strictly increasing, column 578 empty
WARN: A->p (column pointers) not strictly increasing, column 579 empty
WARN: A->p (column pointers) not strictly increasing, column 1152 empty
WARN: A->p (column pointers) not strictly increasing, column 1153 empty
WARN: A->p (column pointers) not strictly increasing, column 1154 empty
WARN: A->p (column pointers) not strictly increasing, column 1155 empty
calculation time: 3.79399991035 [sec]
x is  None


TypeError: 'NoneType' object has no attribute '__getitem__'

In [123]:
print 24*6

144
